# ARBOLES DE DECISION

En primer lugar, importamos las librerías que vamos a necesitar

In [1]:
import pandas as pd
import math
from fractions import Fraction 
import numpy as np

Definimos la función que calcula la entropia

In [2]:
def cantidad_informacion(prob_xi):
    return -math.log(prob_xi,2)

In [3]:
def entropia (problist):
   ci_ponderada=[]
   for prob_xi in problist.values():
       ci_ponderada.append(prob_xi*(cantidad_informacion(prob_xi)))
   return np.sum(ci_ponderada)

Leemos los datos de entrenamiento

In [4]:
df=pd.read_csv("/home/sukete/python/ml_clases/ejemplo_arbol_decision.csv",delimiter='|')
df.head()

,tiempo,horario,compañia,situacion_economica,plan,salida
0,lluvia,dia,amigos,buena,raciones,si
1,sol,dia,amigos,regular,raciones,si
2,sol,dia,amigos,regular,tapas,si
3,sol,noche,amigos,mala,tapas,si
4,sol,noche,amigos,regular,tapas,si


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tiempo               22 non-null     object
 1   horario              22 non-null     object
 2   compañia             22 non-null     object
 3   situacion_economica  22 non-null     object
 4   plan                 22 non-null     object
 5   salida               22 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB


## Determinamos los atributos y sus valores únicos

In [6]:
for attr in df.columns.values:
    print(attr+":"+str(df[attr].unique()))

tiempo:['lluvia' 'sol']
horario:['dia' 'noche']
compañia:['amigos' 'pareja']
situacion_economica:['buena' 'regular' 'mala']
plan:['raciones' 'tapas' 'restaurante']
salida:['si' 'no']


## Comenzamos a ejecutar el algoritmo

En primer lugar, calculamos la entropia total del conjunto de datos

In [7]:
def entropia_total (df):
    problist={}
    tam_muestra=len(df['salida'])
    for suceso in df['salida']:
      if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
      else:    
            problist[suceso]=Fraction(1,tam_muestra)
    return entropia(problist)
ES=entropia_total(df)
print ("La entropia E(S) es "+str(ES))

La entropia E(S) es 0.7732266742876346


Calculamos el peso de cada atributo

In [8]:
S={}
for attr in df.columns.values:
    problist={}
    tam_muestra=len(df[attr])
    for suceso in df[attr]:
        if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
        else:    
            problist[suceso]=Fraction(1,tam_muestra)
    S[attr]=problist
print(S)

{'tiempo': {'lluvia': Fraction(9, 22), 'sol': Fraction(13, 22)}, 'horario': {'dia': Fraction(9, 22), 'noche': Fraction(13, 22)}, 'compañia': {'amigos': Fraction(13, 22), 'pareja': Fraction(9, 22)}, 'situacion_economica': {'buena': Fraction(4, 11), 'regular': Fraction(9, 22), 'mala': Fraction(5, 22)}, 'plan': {'raciones': Fraction(5, 22), 'tapas': Fraction(4, 11), 'restaurante': Fraction(9, 22)}, 'salida': {'si': Fraction(17, 22), 'no': Fraction(5, 22)}}


Ahora calculamos el ratio de ganacia de la información para el atributo 'tiempo'

In [9]:
Ent={}
for valor in df['tiempo'].unique():
 problist={}
 tam_muestra=len(df.loc[df['tiempo']==valor]['salida'])
 for suceso in df.loc[df['tiempo']==valor]['salida']:
    if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
    else:    
            problist[suceso]=Fraction(1,tam_muestra)
 print("S_"+valor+": "+str(problist))
 Ent[valor]=entropia(problist)
for valor in df['tiempo'].unique():
 print ("La entropia E(S_"+valor+"): "+str(Ent[valor]))

S_lluvia: {'si': Fraction(2, 3), 'no': Fraction(1, 3)}
S_sol: {'si': Fraction(11, 13), 'no': Fraction(2, 13)}
La entropia E(S_lluvia): 0.9182958340544896
La entropia E(S_sol): 0.6193821946787638


In [10]:
Ganancia=ES-S['tiempo']['lluvia']*Ent['lluvia']-S['tiempo']['sol']*Ent['sol']
print ("El ratio de ganancia de informacion del atributo tiempo es:" + str(Ganancia))

El ratio de ganancia de informacion del atributo tiempo es:0.03156162713698296


Ahora calculamos el valor de la ganancia de información de cada atributo

In [12]:
for attr in df.columns[:-1].values:
 Ent={}
 Ganancia=ES
 for valor in df[attr].unique():
  problist={}
  tam_muestra=len(df.loc[df[attr]==valor]['salida'])
  for suceso in df.loc[df[attr]==valor]['salida']:
    if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
    else:    
            problist[suceso]=Fraction(1,tam_muestra)
  Ganancia=Ganancia-S[attr][valor]*entropia(problist)
 print ("El ratio de ganancia de informacion del atributo "+attr+" es:" + str(Ganancia))

El ratio de ganancia de informacion del atributo tiempo es:0.03156162713698296
El ratio de ganancia de informacion del atributo horario es:0.04114865902244358
El ratio de ganancia de informacion del atributo compañia es:0.041148659022443634
El ratio de ganancia de informacion del atributo situacion_economica es:0.14901787743077435
El ratio de ganancia de informacion del atributo plan es:0.20371190097032488


El atributo que mayor ganacia aporta es 'plan', por lo que lo vamos a colocar como primer nodo del arbol

### Repetimos el proceso para cada uno de los nodos generados en la primera iteracion

Vemos como seguiria el proceso para el nodo a partir de valor del campo 'plan' igual a tapas

In [13]:
df_l1_tapas=df.loc[df['plan']=='tapas'].drop(['plan'],axis=1)
df_l1_tapas.head()

,tiempo,horario,compañia,situacion_economica,salida
2,sol,dia,amigos,regular,si
3,sol,noche,amigos,mala,si
4,sol,noche,amigos,regular,si
8,sol,dia,pareja,mala,si
18,lluvia,dia,amigos,regular,si


Calculamos el ratio de ganancia de informacion para el resto de los atributos

In [14]:
for attr in df_l1_tapas.columns[:-1].values:
 Ent={}
 Ganancia=entropia_total(df_l1_tapas)
 for valor in df_l1_tapas[attr].unique():
  problist={}
  tam_muestra=len(df_l1_tapas.loc[df[attr]==valor]['salida'])
  for suceso in df_l1_tapas.loc[df_l1_tapas[attr]==valor]['salida']:
    if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
    else:    
            problist[suceso]=Fraction(1,tam_muestra)
  Ganancia=Ganancia-S[attr][valor]*entropia(problist)
 print ("La gananancia de informacion del atributo "+attr+" es:" + str(Ganancia))

La gananancia de informacion del atributo tiempo es:0.0
La gananancia de informacion del atributo horario es:0.0
La gananancia de informacion del atributo compañia es:0.0
La gananancia de informacion del atributo situacion_economica es:0.0


Ahora, de forma general, calculamos la ganancia de información para cada uno de los nodos del siguiente nivel del arbol

In [16]:
df_l1={}
for nodo in df['plan'].unique():
    print ("Calculo Gancia de información para el nodo "+nodo)
    df_l1[nodo]=df.loc[df['plan']==nodo].drop(['plan'],axis=1)
    print (df_l1[nodo])
    for attr in df_l1[nodo].columns[:-1].values:
      Ent={}
      Ganancia=entropia_total(df_l1[nodo])
      S={}
      problist={}
      tam_muestra=len(df_l1[nodo][attr])
      for suceso in df_l1[nodo][attr]:
           if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
           else:    
            problist[suceso]=Fraction(1,tam_muestra)
      S[attr]=problist
      for valor in df_l1[nodo][attr].unique():
       problist={}
       tam_muestra=len(df_l1[nodo].loc[df_l1[nodo][attr]==valor]['salida'])
       for suceso in df_l1[nodo].loc[df_l1[nodo][attr]==valor]['salida']:
           if suceso in problist:
            problist[suceso]=problist[suceso]+Fraction(1,tam_muestra)
           else:    
            problist[suceso]=Fraction(1,tam_muestra)
       Ganancia=Ganancia-S[attr][valor]*entropia(problist)
      print ("La ganancia de informacion del atributo "+attr+" es:" + str(Ganancia))

Calculo Gancia de información para el nodo raciones
    tiempo horario compañia situacion_economica salida
0   lluvia     dia   amigos               buena     si
1      sol     dia   amigos             regular     si
7   lluvia   noche   pareja                mala     no
11     sol   noche   pareja             regular     si
12  lluvia   noche   amigos             regular     si
La ganancia de informacion del atributo tiempo es:0.17095059445466865
La ganancia de informacion del atributo horario es:0.17095059445466865
La ganancia de informacion del atributo compañia es:0.3219280948873623
La ganancia de informacion del atributo situacion_economica es:0.7219280948873623
Calculo Gancia de información para el nodo tapas
    tiempo horario compañia situacion_economica salida
2      sol     dia   amigos             regular     si
3      sol   noche   amigos                mala     si
4      sol   noche   amigos             regular     si
8      sol     dia   pareja                mala     si


In [17]:
df_l2_raciones=df_l1['raciones'].loc[df_l1['raciones']['situacion_economica']=='mala'].drop(['situacion_economica'],axis=1)
df_l2_raciones.head()

,tiempo,horario,compañia,salida
7,lluvia,noche,pareja,no


In [18]:
df_l2_raciones=df_l1['raciones'].loc[df_l1['raciones']['situacion_economica']=='regular'].drop(['situacion_economica'],axis=1)
df_l2_raciones.head()

,tiempo,horario,compañia,salida
1,sol,dia,amigos,si
11,sol,noche,pareja,si
12,lluvia,noche,amigos,si


In [19]:
df_l2_raciones=df_l1['raciones'].loc[df_l1['raciones']['situacion_economica']=='buena'].drop(['situacion_economica'],axis=1)
df_l2_raciones.head()

,tiempo,horario,compañia,salida
0,lluvia,dia,amigos,si


In [20]:
df_l2_restaurante=df_l1['restaurante'].loc[df_l1['restaurante']['compañia']=='pareja'].drop(['compañia'],axis=1)
df_l2_restaurante.head()

,tiempo,horario,situacion_economica,salida
9,sol,dia,buena,si
10,sol,noche,buena,si
13,lluvia,noche,regular,si
14,lluvia,noche,buena,si


In [21]:
df_l2_restaurante=df_l1['restaurante'].loc[df_l1['restaurante']['compañia']=='amigos'].drop(['compañia'],axis=1)
df_l2_restaurante.head()

,tiempo,horario,situacion_economica,salida
5,lluvia,noche,mala,no
6,sol,noche,mala,no
15,lluvia,noche,buena,no
16,sol,noche,buena,si
17,sol,dia,regular,no
